In [2]:
!pip install owlready2
!pip install rdflib
!pip install OpenAI

In [4]:
from rdflib import Graph
from openai import OpenAI
from owlready2 import get_ontology, onto_path
import io

In [3]:
system_prompt = """
You are an expert in ontology engineering. Generate an OWL ontology based on the following domain description:
Define classes, data properties, and object properties.
Include domain and range for each property.
Provide the output in OWL (XML) format."""

client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

user_prompt = """A healthcare domain where:

Patients have a name, age, and gender.
Patients can have conditions and are treated by doctors.
Conditions have a name.
Doctors have a name and specialty.
"""

completion = client.chat.completions.create(
model="model-identifier",
messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
    ],
    temperature=0.7,
)

content = completion.choices[0].message.content
clean_content = content.split("</think>\n\n")[-1] 

print(clean_content)

In [16]:
ontology = get_ontology("ontology.owl").load()

if ontology is None:
    print("Error: The ontology was not loaded correctly.")
else:
    print("Ontology loaded successfully:", ontology)

print("\nDetected classes:")
for cls in ontology.classes():
    print(f"- {cls.name}")
if not list(ontology.classes()):
    print("No classes detected in the ontology.")

print("\nDetected properties:")
for prop in ontology.properties():
    print(f"- {prop.name}")
if not list(ontology.properties()):
    print("No properties detected in the ontology.")

# Print ontology structure
print("Classes:")
for cls in ontology.classes():
    print(cls)

print("\nProperties:")
for prop in ontology.properties():
    print(f"{prop}: Domain={prop.domain}, Range={prop.range}")


Ontology loaded successfully: get_ontology("http://example.org/healthcare#")

Detected classes:
- Patient
- Condition
- Doctor

Detected properties:
- patientName
- age
- gender
- conditionName
- specialty
- hasCondition
- treats
Classes:
ontology.Patient
ontology.Condition
ontology.Doctor

Properties:
ontology.patientName: Domain=[ontology.Patient], Range=[<class 'str'>]
ontology.age: Domain=[ontology.Patient], Range=[<class 'int'>]
ontology.gender: Domain=[ontology.Patient], Range=[<class 'str'>]
ontology.conditionName: Domain=[ontology.Condition], Range=[<class 'str'>]
ontology.specialty: Domain=[ontology.Doctor], Range=[<class 'str'>]
ontology.hasCondition: Domain=[ontology.Patient], Range=[ontology.Condition]
ontology.treats: Domain=[ontology.Doctor], Range=[ontology.Patient]


In [18]:
ontology_uri = "http://example.org/ontology.owl"
onto_path.append(".")
ontology = get_ontology(ontology_uri).load()
graph = ontology.world.as_rdflib_graph()

# Serialise to Turtle
ontology_buffer = io.BytesIO()
graph.serialize(destination=ontology_buffer, format="turtle", base=None)
ontology_scheme = ontology_buffer.getvalue().decode("utf-8")

print(f"\nTurtle serialization length: {len(ontology_scheme)}")
print(ontology_scheme[:1000]) 


Turtle serialization length: 3309
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<./ontology.owl> a owl:Ontology .

<http://anonymous> a owl:Ontology .

<http://example.org/healthcare> a owl:Ontology,
        owl:Ontology ;
    rdfs:comment "A healthcare domain ontology defining patients, conditions, and doctors."^^xsd:string,
        "A healthcare domain ontology defining patients, conditions, and doctors."^^xsd:string .

<http://example.org/healthcare#age> a owl:DatatypeProperty,
        owl:DatatypeProperty ;
    rdfs:comment "Age of the patient"^^xsd:string,
        "Age of the patient"^^xsd:string ;
    rdfs:domain <http://example.org/healthcare#Patient>,
        <http://example.org/healthcare#Patient> ;
    rdfs:range xsd:integer,
        xsd:integer .

<http://example.org/healthcare#conditionName> a owl:DatatypeProperty,
        owl:DatatypeProperty ;
    rdfs:comment "N

In [26]:
# Function to generate RDF triples using LLM
text_input= "Patient John Doe, aged 45, was diagnosed with diabetes and treated by Dr. Smith."
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

completion = client.chat.completions.create(
    model="model-identifier",
    messages=[
    {"role": "system", "content": "Extract RDF triples from the following text in Turtle format, adhering to the ontology: - Patient: patientName, age, gender, hasCondition. - Doctor: hasName, specialty. - Condition: conditionName. Ontology:" + ontology_scheme},
    {"role": "user", "content": "Text: " + text_input + ". Generate RDF triples in Turtle format."}
    ],
    temperature=0.7,
)

content = completion.choices[0].message.content
clean_content = content.split("</think>\n\n")[-1] 

print(clean_content)

Here is the RDF representation based on the given text:

```turtle
_:patient1 a <http://example.org/healthcare#Patient>;
    patientName "John Doe";
    age <http://example.org/healthcare#age> 45;
    hasCondition <http://example.org/healthcare#diabetes>.

<http://example.org/healthcare#diabetes> a rdfs:Resource;
    rdfs:label "Diabetes"^^xsd:string.

_:doctor1 a <http://example.org/healthcare#Doctor>;
    hasName "Dr. Smith";
    <http://example.org/healthcare#specialty> "General Practice".
```

This output includes all the necessary triples based on the provided text, adhering to the defined ontology.


In [ ]:
def validate_rdf(rdf_data, ontology):
    g = Graph()
    g.parse(data=rdf_data, format="turtle")
    errors = []
    for s, p, o in g:
        prop_name = p.split("#")[-1]
        ontology_prop = getattr(ontology, prop_name, None)
        
        if not ontology_prop:
            errors.append(f"Property ‘{prop_name}’ not found in ontology.")
        elif isinstance(o, str) and xsd:string not in ontology_prop.range:
            errors.append(f"Range Error: {p} expects {ontology_prop.range}, but found a string.")
    return errors


In [ ]:
def refine_rdf(rdf_data, feedback):
    refinement_prompt = f"""
    The following RDF output has errors:
    {rdf_data}
    Errors: {feedback}
    Refine the RDF triples to fix these issues while adhering to the ontology schema.
    """
    client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

    completion = client.chat.completions.create(
        model="model-identifier",
        messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "Text: " + refinement_prompt}
        ],
        temperature=0.7,
    )

    return response.content